In [ ]:
import dcarte
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

import pyarrow as pa
import pyarrow.parquet as pq

import seaborn as sns
pd.set_option('display.max_columns', None)

In [ ]:
ids = np.load('ids_seven_days_each_five_bimonth.npy')
len(ids)

In [ ]:
patient = dcarte.load('Patients','lookup',reload=False)
patient['participant_id'] = patient['id'].apply(lambda x:x[:5])
patient = patient[patient['participant_id'].isin(ids)]
#patient

In [ ]:
dyads = pd.read_excel('dyads_unprotected.xlsx')

dyads['participant_id'] = dyads['Research ID'].apply(lambda x:x[:5])
dyads = dyads[dyads['participant_id'].isin(ids)]
dyads

In [ ]:
demographics = pd.read_csv('redcap_16_nov_2023/Demographics_MinderHealthManageme_DATA_LABELS_2023-11-16_1127.csv')
#demographics.columns.to_list()

In [ ]:
demographics = demographics[['Dyad number', 'Year of birth PwD',
 'Gender PwD','Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))',
 'Clinical Diagnosis (choice=Lewy Body Dementia)',
 'Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
 'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))',
 'Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
 'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
 'Clinical Diagnosis (choice=Vascular dementia)',
 'Clinical Diagnosis (choice=Mild cognitive impairment (MCI))',
 'Clinical Diagnosis (choice=Young-onset dementia)',
 'Clinical Diagnosis (choice=Alcohol-related brain damage)',
 'Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)',
 'Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)',
 'Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
 'Clinical Diagnosis (choice=Traumatic Brain Injury)',
 'Clinical Diagnosis (choice=No dementia)',
 'Clinical Diagnosis (choice=Stroke)',
 'Clinical Diagnosis (choice=Frailty)']]
#diagnosis, imaging results and regions affected, comorbidities, smoke, drink, house style, mobility, pets
demographics  = demographics.rename(columns={"Dyad number": "DYAD"})
demographics.head(3)

In [ ]:
alzheimers = ['Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))','Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))','Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))']
demographics['alzheimers'] = demographics[alzheimers].ne('Unchecked').any(axis=1)
dementias =['Clinical Diagnosis (choice=Lewy Body Dementia)','Clinical Diagnosis (choice=Dementia in Parkinsons Disease)','Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))', 'Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
 'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))','Clinical Diagnosis (choice=Vascular dementia)','Clinical Diagnosis (choice=Young-onset dementia)']
demographics['dementias'] = demographics[dementias].ne('Unchecked').any(axis=1)
demographics['alzheimer_and_dementia'] = demographics[['Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)','Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)']].ne('Unchecked').any(axis=1)
demographics['mci'] = demographics[['Clinical Diagnosis (choice=Mild cognitive impairment (MCI))']].ne('Unchecked').any(axis=1)

demographics

In [ ]:
merge1 = dyads.merge(demographics, on='DYAD')
merge1 = merge1[['participant_id', 'Year of birth PwD',
       'Gender PwD',
       'Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))',
       'Clinical Diagnosis (choice=Lewy Body Dementia)',
       'Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
       'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))',
       'Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
       'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
       'Clinical Diagnosis (choice=Vascular dementia)',
       'Clinical Diagnosis (choice=Mild cognitive impairment (MCI))',
       'Clinical Diagnosis (choice=Young-onset dementia)',
       'Clinical Diagnosis (choice=Alcohol-related brain damage)',
       'Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)',
       'Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)',
       'Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
       'Clinical Diagnosis (choice=Traumatic Brain Injury)',
       'Clinical Diagnosis (choice=No dementia)',
       'Clinical Diagnosis (choice=Stroke)',
       'Clinical Diagnosis (choice=Frailty)', 'alzheimers', 'dementias',
       'alzheimer_and_dementia', 'mci']]


In [ ]:
merge1.loc[merge1['alzheimers'] ==True, 'Diagnosis'] = 'Alzheimers'
merge1.loc[merge1['dementias'] ==True, 'Diagnosis'] = 'Dementias'
merge1.loc[merge1['mci'] ==True, 'Diagnosis'] = 'MCI'
merge1.loc[(merge1['alzheimer_and_dementia'] ==True), 'Diagnosis'] = 'Alzheimers and Dementias'
merge1.loc[(merge1['alzheimers'] ==True)&(merge1['dementias'] ==True), 'Diagnosis'] = 'Alzheimers and Dementias'

merge1['alzheimer_diagnosis'] ='no'
merge1.loc[(merge1['Diagnosis'] =='Alzheimers and Dementias')|(merge1['Diagnosis']=='Alzheimers'),'alzheimer_diagnosis'] = 'yes'
merge1.loc[(merge1['Diagnosis'] =='Unknown'),'alzheimer_diagnosis'] = 'unknown'


In [ ]:
merge1[merge1['participant_id']=='TbQch']

In [ ]:
merge1 = merge1[['participant_id', 'Year of birth PwD', 'Gender PwD','Diagnosis','alzheimer_diagnosis','Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))','Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))','Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))','Clinical Diagnosis (choice=Lewy Body Dementia)','Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
       'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))','Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
       'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))', 'Clinical Diagnosis (choice=Vascular dementia)','Clinical Diagnosis (choice=Mild cognitive impairment (MCI))','Clinical Diagnosis (choice=Young-onset dementia)',
       'Clinical Diagnosis (choice=Alcohol-related brain damage)','Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)','Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)','Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
       'Clinical Diagnosis (choice=Traumatic Brain Injury)','Clinical Diagnosis (choice=No dementia)','Clinical Diagnosis (choice=Stroke)','Clinical Diagnosis (choice=Frailty)']]
merge1

merge2[merge2.isna().any(axis=1)]

In [ ]:
merge1.to_excel("redcap_demographics_diagnosis_nov16.xlsx")

In [ ]:
merge1.groupby(by=['alzheimer_diagnosis']).count()